In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import sklearn as sk
data = pd.read_csv('/content/drive/MyDrive/breast cancer.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import dataset, pandas and sklearn library  

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

data["diagnosis"] = label_encoder.fit_transform(data["diagnosis"])
print(data.head())

         id  diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302          1        17.99         10.38          122.80     1001.0   
1    842517          1        20.57         17.77          132.90     1326.0   
2  84300903          1        19.69         21.25          130.00     1203.0   
3  84348301          1        11.42         20.38           77.58      386.1   
4  84358402          1        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  radius_worst  texture_worst  perimeter_worst  ar

#encode target label from (m,b) to (0,1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
print(normalized_data.head())

         id  diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0  0.000915        1.0     0.521037      0.022658        0.545989   0.363733   
1  0.000915        1.0     0.643144      0.272574        0.615783   0.501591   
2  0.092495        1.0     0.601496      0.390260        0.595743   0.449417   
3  0.092547        1.0     0.210090      0.360839        0.233501   0.102906   
4  0.092559        1.0     0.629893      0.156578        0.630986   0.489290   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0         0.593753          0.792037        0.703140             0.731113   
1         0.289880          0.181768        0.203608             0.348757   
2         0.514309          0.431017        0.462512             0.635686   
3         0.811321          0.811361        0.565604             0.522863   
4         0.430351          0.347893        0.463918             0.518390   

   ...  radius_worst  texture_worst  perimeter_worst  ar

#data normalizing for all columns

In [ ]:
from sklearn.model_selection import train_test_split

X = normalized_data.drop(['diagnosis','id'], axis=1)

y = normalized_data['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#split data set to train and test part and droping id label(unused)

In [ ]:
#correlation matrix for feature selection
correlation_matrix = X_train.corr()

print("Correlation Matrix:")
print(correlation_matrix)

threshold = 0.90

# droping non important features which are corelated more than 0.9
important_features_corr= []
for col in correlation_matrix:
    correlated_features = correlation_matrix[abs(correlation_matrix[col]) > threshold].index.tolist()
    correlated_features.remove(col)
    important_features_corr.extend(correlated_features)

important_features_cr = list(set(important_features_corr))


print("Important Features:")
for feature in important_features_corr:
    print(feature)

Correlation Matrix:
                         radius_mean  texture_mean  perimeter_mean  area_mean  \
radius_mean                 1.000000      0.306881        0.997850   0.986609   
texture_mean                0.306881      1.000000        0.313124   0.300314   
perimeter_mean              0.997850      0.313124        1.000000   0.985985   
area_mean                   0.986609      0.300314        0.985985   1.000000   
smoothness_mean             0.173051      0.009475        0.210092   0.180709   
compactness_mean            0.477986      0.229954        0.529721   0.471296   
concavity_mean              0.647311      0.282160        0.688251   0.659793   
concave points_mean         0.812377      0.286231        0.842081   0.814273   
symmetry_mean               0.150386      0.108045        0.185885   0.152679   
fractal_dimension_mean     -0.335575     -0.079010       -0.286307  -0.305321   
radius_se                   0.666954      0.249347        0.680739   0.722728   
texture_

#Important Features by correlation matrix:
perimeter_mean
area_mean
radius_worst
perimeter_worst
area_worst
texture_worst
radius_mean
area_mean
radius_worst
perimeter_worst
area_worst
radius_mean
perimeter_mean
radius_worst
perimeter_worst
area_worst
concave points_mean
concavity_mean
concave points_worst
perimeter_se
area_se
radius_se
area_se
radius_se
perimeter_se
radius_mean
perimeter_mean
area_mean
perimeter_worst
area_worst
texture_mean
radius_mean
perimeter_mean
area_mean
radius_worst
area_worst
radius_mean
perimeter_mean
area_mean
radius_worst
perimeter_worst
concave points_mean

#As we test for correlation matrix to select features,we did not receive acceptable information from it.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler


# Random Forest model
model = RandomForestClassifier()
model.fit(X_train, y_train)
importances = model.feature_importances_

feature_names = X_train.columns[:-1]
for feature_name, importance in zip(feature_names, importances):
    print(f"{feature_name}: {importance}")


threshold = 0.04
important_features = [feature_name for feature_name, importance in zip(feature_names, importances) if importance > threshold]
print("Important features:", important_features)

radius_mean: 0.027057477555637147
texture_mean: 0.015052316278608114
perimeter_mean: 0.04164975567681995
area_mean: 0.07249786342294766
smoothness_mean: 0.006377923675356394
compactness_mean: 0.010460767182804807
concavity_mean: 0.061950115975755206
concave points_mean: 0.11493685814496725
symmetry_mean: 0.003945110793827371
fractal_dimension_mean: 0.004598170939693784
radius_se: 0.0069098212750469375
texture_se: 0.0029742514080259423
perimeter_se: 0.011985649962370691
area_se: 0.024169854704224992
smoothness_se: 0.0029600909410114314
compactness_se: 0.005104339743554149
concavity_se: 0.011482392234814554
concave points_se: 0.0053692158786083225
symmetry_se: 0.004425241225384424
fractal_dimension_se: 0.003809914045738769
radius_worst: 0.12910884268776365
texture_worst: 0.018798791101164926
perimeter_worst: 0.09759524749166443
area_worst: 0.08837302761114744
smoothness_worst: 0.01680119143146258
compactness_worst: 0.01736138421160472
concavity_worst: 0.0479009173746022
concave points_wo

#we are using random forest model to find most important features with 0.05 threshold and some features left for us.
#some of them are related to same parameter but they're diffrent. after that we set mlp model with 2 diffrent learning rate and epochs.
#learning rate: 0.01 and 0.001
#epochs :  20 and 50
#we use 64 hidden Neurons
#for the breast cancer case, we should have the best recall we can get because false negative data is more important than false positive.

In [ ]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score


input_dim = len(important_features)
hidden_dim = 64
output_dim = 1
learning_rate = 0.001
epochs = 20

selected_X_train = X_train[important_features]
selected_X_test = X_test[important_features]



model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(hidden_dim, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='sigmoid')
])

# runing model by accuracy metric
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(selected_X_train, y_train, epochs=epochs, validation_split=0.2)

#predicting for test data set
y_pred_probs = model.predict(selected_X_test)
y_pred = (y_pred_probs > 0.5).astype(int)


#printing confusion matrix , recall and f1score
confusion_mat = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = 2*precision*recall/(precision+recall)

print("Confusion Matrix:")
print(confusion_mat)
print("Precision:", precision)
print("Recall:", recall)
print("f1score:", f1)

Epoch 1/20
12/12 [==============================] - 1s 23ms/step - loss: 0.6825 - accuracy: 0.6264 - val_loss: 0.6596 - val_accuracy: 0.6923
Epoch 2/20
12/12 [==============================] - 0s 7ms/step - loss: 0.6474 - accuracy: 0.7445 - val_loss: 0.6241 - val_accuracy: 0.8352
Epoch 3/20
12/12 [==============================] - 0s 6ms/step - loss: 0.6089 - accuracy: 0.8242 - val_loss: 0.5795 - val_accuracy: 0.9231
Epoch 4/20
12/12 [==============================] - 0s 5ms/step - loss: 0.5640 - accuracy: 0.8929 - val_loss: 0.5234 - val_accuracy: 0.9670
Epoch 5/20
12/12 [==============================] - 0s 6ms/step - loss: 0.5034 - accuracy: 0.8984 - val_loss: 0.4544 - val_accuracy: 0.9670
Epoch 6/20
12/12 [==============================] - 0s 5ms/step - loss: 0.4343 - accuracy: 0.9121 - val_loss: 0.3828 - val_accuracy: 0.9341
Epoch 7/20
12/12 [==============================] - 0s 5ms/step - loss: 0.3668 - accuracy: 0.9231 - val_loss: 0.3178 - val_accuracy: 0.9341
Epoch 8/20
12/12 [=

In [ ]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score


input_dim = len(important_features)
hidden_dim = 64
output_dim = 1
learning_rate = 0.01
epochs = 50


selected_X_train = X_train[important_features]
selected_X_test = X_test[important_features]



model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(hidden_dim, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='sigmoid')
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(selected_X_train, y_train, epochs=epochs, validation_split=0.2)


y_pred_probs = model.predict(selected_X_test)
y_pred = (y_pred_probs > 0.5).astype(int)


confusion_mat = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = 2*precision*recall/(precision+recall)

print("Confusion Matrix:")
print(confusion_mat)
print("Precision:", precision)
print("Recall:", recall)
print("f1score:", f1)

Epoch 1/50
12/12 [==============================] - 1s 21ms/step - loss: 0.5710 - accuracy: 0.7225 - val_loss: 0.3754 - val_accuracy: 0.9231
Epoch 2/50
12/12 [==============================] - 0s 7ms/step - loss: 0.2380 - accuracy: 0.9313 - val_loss: 0.1428 - val_accuracy: 0.9451
Epoch 3/50
12/12 [==============================] - 0s 6ms/step - loss: 0.1794 - accuracy: 0.9286 - val_loss: 0.2582 - val_accuracy: 0.9121
Epoch 4/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2450 - accuracy: 0.9148 - val_loss: 0.1770 - val_accuracy: 0.9231
Epoch 5/50
12/12 [==============================] - 0s 6ms/step - loss: 0.1831 - accuracy: 0.9231 - val_loss: 0.1568 - val_accuracy: 0.9231
Epoch 6/50
12/12 [==============================] - 0s 5ms/step - loss: 0.1658 - accuracy: 0.9368 - val_loss: 0.1794 - val_accuracy: 0.9011
Epoch 7/50
12/12 [==============================] - 0s 5ms/step - loss: 0.1491 - accuracy: 0.9396 - val_loss: 0.1325 - val_accuracy: 0.9451
Epoch 8/50
12/12 [=

In [ ]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score


input_dim = len(important_features)
hidden_dim = 64
output_dim = 1
learning_rate = 0.01
epochs = 20

selected_X_train = X_train[important_features]
selected_X_test = X_test[important_features]



model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(hidden_dim, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='sigmoid')
])

# کامپایل مدل
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])


history = model.fit(selected_X_train, y_train, epochs=epochs, validation_split=0.2)


y_pred_probs = model.predict(selected_X_test)
y_pred = (y_pred_probs > 0.5).astype(int)



confusion_mat = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = 2*precision*recall/(precision+recall)

print("Confusion Matrix:")
print(confusion_mat)
print("Precision:", precision)
print("Recall:", recall)
print("f1score:", f1)

Epoch 1/20
12/12 [==============================] - 2s 91ms/step - loss: 0.5865 - accuracy: 0.7308 - val_loss: 0.3695 - val_accuracy: 0.9560
Epoch 2/20
12/12 [==============================] - 0s 9ms/step - loss: 0.2528 - accuracy: 0.9368 - val_loss: 0.1404 - val_accuracy: 0.9560
Epoch 3/20
12/12 [==============================] - 0s 9ms/step - loss: 0.1726 - accuracy: 0.9341 - val_loss: 0.1520 - val_accuracy: 0.9451
Epoch 4/20
12/12 [==============================] - 0s 7ms/step - loss: 0.1698 - accuracy: 0.9286 - val_loss: 0.1922 - val_accuracy: 0.9011
Epoch 5/20
12/12 [==============================] - 0s 9ms/step - loss: 0.1894 - accuracy: 0.9258 - val_loss: 0.1444 - val_accuracy: 0.9451
Epoch 6/20
12/12 [==============================] - 0s 7ms/step - loss: 0.1576 - accuracy: 0.9368 - val_loss: 0.1568 - val_accuracy: 0.9231
Epoch 7/20
12/12 [==============================] - 0s 7ms/step - loss: 0.1537 - accuracy: 0.9368 - val_loss: 0.1785 - val_accuracy: 0.9011
Epoch 8/20
12/12 [=

In [ ]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score


input_dim = len(important_features)
hidden_dim = 64
output_dim = 1
learning_rate = 0.001
epochs = 50


selected_X_train = X_train[important_features]
selected_X_test = X_test[important_features]



model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_dim, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(hidden_dim, activation='relu'),
    tf.keras.layers.Dense(output_dim, activation='sigmoid')
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])


history = model.fit(selected_X_train, y_train, epochs=epochs, validation_split=0.2)


y_pred_probs = model.predict(selected_X_test)
y_pred = (y_pred_probs > 0.5).astype(int)



confusion_mat = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = 2*precision*recall/(precision+recall)

print("Confusion Matrix:")
print(confusion_mat)
print("Precision:", precision)
print("Recall:", recall)
print("f1score:", f1)

Epoch 1/50
12/12 [==============================] - 1s 20ms/step - loss: 0.6895 - accuracy: 0.6978 - val_loss: 0.6637 - val_accuracy: 0.9121
Epoch 2/50
12/12 [==============================] - 0s 7ms/step - loss: 0.6466 - accuracy: 0.8709 - val_loss: 0.6183 - val_accuracy: 0.9231
Epoch 3/50
12/12 [==============================] - 0s 5ms/step - loss: 0.5982 - accuracy: 0.9093 - val_loss: 0.5621 - val_accuracy: 0.9451
Epoch 4/50
12/12 [==============================] - 0s 5ms/step - loss: 0.5358 - accuracy: 0.9451 - val_loss: 0.4901 - val_accuracy: 0.9560
Epoch 5/50
12/12 [==============================] - 0s 6ms/step - loss: 0.4597 - accuracy: 0.9396 - val_loss: 0.4123 - val_accuracy: 0.9231
Epoch 6/50
12/12 [==============================] - 0s 5ms/step - loss: 0.3853 - accuracy: 0.9313 - val_loss: 0.3399 - val_accuracy: 0.9451
Epoch 7/50
12/12 [==============================] - 0s 5ms/step - loss: 0.3177 - accuracy: 0.9341 - val_loss: 0.2806 - val_accuracy: 0.9451
Epoch 8/50
12/12 [=

#results:
##looks like 0.01 learning rate is too big for good result in 20 and 50 epochs and 0.001 learning rate results are better and make a better recall in 50 epochs.